In [1]:
%%capture
import sys
sys.path.append("../../bus_service_increase")
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

import altair as alt
import calitp_data_analysis.magics
import pandas as pd
import annual_ridership_module

from IPython.display import display, HTML, Markdown

from bus_service_utils import chart_utils
from calitp_data_analysis import calitp_color_palette as cp
from update_vars import GCS_FILE_PATH, PUBLIC_FILENAME, YEAR, MONTH

#from shared_utils.rt_dates import MONTH_DICT

#alt.renderers.enable("html")
alt.data_transformers.enable('default', max_rows=None)

WIDTH = 300
HEIGHT = 150

In [2]:
# parameters cell for local
rtpa = "Metropolitan Transportation Commission"

In [3]:
%%capture_parameters
rtpa

{"rtpa": "Metropolitan Transportation Commission"}


# {rtpa}
## Annual Ridership Trends

**Download data from our public [folder](https://console.cloud.google.com/storage/browser/calitp-publish-data-analysis)** by navigating to `ntd_annual_ridership` and selecting a file.

Transit operators/agencies that aubmit annual reports to NTD are included in this report. Reporters that were previously Urban full reporters, but are currently not, may appear. This may result in Reporters showing zero or partial ridership data in the report. 

If a Reporter is not a monthly reporter, or has not reported data since 2018, they will not appear in the report.

Examples: 
- **Reporter A** is an urban full reporter from 2019-2022, then became a reduced reporter for 2023. Reporter A's ridership data will be displayed for 2019-2022 only.
- **Reporter B** is an urban full reporter from 2000-2017, then became a reduced reporter for 2018. Reporter B will be named in the report, but will not display ridership data.
- **Reporter C** was a reduced reporter form 2015-2020, then became an urban full reporter and began submitting monthly ridership data to NTD for 2021. Reporter C's ridership data will be displayed for 2021-present. 

In [4]:
URL = ("https://console.cloud.google.com/storage/"
       "browser/calitp-publish-data-analysis"
      )

display(
    HTML(
        f"""
        <a href={URL}>
        Download the latest month of data: {PUBLIC_FILENAME}</a>
        """
    )
)

In [5]:
#read in data
df = pd.read_parquet(
    f"{GCS_FILE_PATH}annual_ridership_report_data.parquet",
    filters=[("RTPA","==",rtpa)]
)

In [7]:
def sum_by_group(
    df: pd.DataFrame,
    group_cols: list) -> pd.DataFrame:
    """
    since data is now long to begin with, this replaces old sum_by_group, make_long and assemble_long_df functions.
    
    """
    grouped_df = df.groupby(group_cols+
                            ["year"]
                           ).agg({
        "upt":"sum",
        #"vrm":"sum",
        #"vrh":"sum",
        "previous_y_upt":"sum",
        "change_1yr":"sum"
    }
    ).reset_index()
    
    #get %change back
    grouped_df = annual_ridership_module.get_percent_change(grouped_df)
    
    #decimal to whole number
    grouped_df["pct_change_1yr"] = grouped_df["pct_change_1yr"]*100
    
    return grouped_df

In [8]:
def group_by_agency(df):
    """
    Take in the 'by_ageny_long' df and aggregatese by rtpa, and calculates upt % of total. 
    To be used in pie chart
    """
    initial_agg = df.groupby("agency_name").agg(
        total_upt=("upt","sum")
    ).reset_index()
    
    # % total columns
    initial_agg["pct_of_total_upt"] = (initial_agg["total_upt"]/initial_agg["total_upt"].sum())*100
    
    # cleaning data types and rounding
    initial_agg["total_upt"] = initial_agg["total_upt"].astype("int64")
    initial_agg["pct_of_total_upt"] = initial_agg["pct_of_total_upt"].round(decimals=2)
    cleaned_agg = initial_agg.sort_values(by="total_upt", ascending = False)
    
    return cleaned_agg

In [35]:

agency_cols = ["ntd_id", "agency_name", "RTPA"]
mode_cols = ["mode", "RTPA"]
tos_cols = ["service", "RTPA"]
reporter_type_cols = ["reporter_type", "RTPA"]

by_agency_long = sum_by_group(df, agency_cols)
by_mode_long = sum_by_group(df, mode_cols)
by_tos_long = sum_by_group(df, tos_cols)
by_reporter_type_long = sum_by_group(df, reporter_type_cols)

In [10]:
# agg by agency, for pie chart
agency_agg_yr = group_by_agency(by_agency_long)

# total UPT check 
total_upt = by_agency_long["upt"].sum()
agency_count = by_agency_long["agency_name"].nunique()

In [31]:
def make_line_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
    title: str,
) -> alt.Chart:
    df = df[df[y_col] > 0].dropna(subset = y_col)

    #x_label = [i for i in df.report_year.unique() if 
    #           any(substring in i for substring in 
    #               ["-01", "-06"])
    #          ]    
    
    chart = (alt.Chart(df)
         .mark_line(point=True)
         .encode(
             x = alt.X("year:O", 
                       #axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(
                 y_col, 
                 title = y_col
             ),
             color = alt.Color(color_col,
                               #title = "", 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               )),
             tooltip = ["year", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(y="independent")
    ).properties(
        title = title
    ).interactive()
        
    return chart

In [33]:
def make_bar_chart(
    df: pd.DataFrame, 
    y_col: str,
    color_col: str,
    title: str,
) -> alt.Chart:
    
    def short_label(word):
        shorten_dict = {
            "change_1yr": "Change",
            "pct_change_1yr":"Change",
        }
        return shorten_dict[word]
    
    # For change column, we are missing everything prior to 2023 
    #df = df.dropna(subset = y_col)
    
    #need flag for y_col >,<, 0, missing?
    #count function to how many agencies fall in those categories, then look at those agencies
    #present table
    
    #x_label = [i for i in df.report_year.unique() if 
    #           any(substring in i for substring in 
    #               ["-01", "-03", "-06", "-09"])
    #          ]
    
    chart = (alt.Chart(df)
         .mark_bar()
         .encode(
             x = alt.X("year:O", 
                       #axis=alt.Axis(values = x_label), 
                       title = "Date"
                      ),
             y = alt.Y(
                 y_col, 
                 title = y_col
             ),
             color = alt.Color(color_col, title = "", 
                              scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                              )),
             tooltip = ["year", y_col, color_col, "RTPA"]
         ).properties(width = WIDTH, height = HEIGHT)
         .facet(color_col, columns=2, title = "")
         .resolve_scale(x="shared", 
                        y="independent")
    ).properties(
        title = title
    ).interactive()
    
    return chart

In [13]:
def make_pie_chart(df,
             col,
             color_col):
    
    pie = alt.Chart(df).mark_arc(radius = 150).encode(
        theta=col,
        color=color_col,
        tooltip = ["agency_name","total_upt","pct_of_total_upt"]
    ).properties(
        title=f"Total Annual Unlinked Passenger Trips per Reporter in RTPA since 2018",

    )

    return pie

In [14]:
#simple bar chart for total agencies and UPT
def total_upt_chart(df: pd.DataFrame, x_col:str, y_col:str,tool_tip:list):
    bar_chart = alt.Chart(df).mark_bar().encode(
    x=alt.X(x_col).sort("-y"),
    y=alt.Y(y_col),
    tooltip = tool_tip,
    color = alt.Color(x_col, title = "", 
                               scale = alt.Scale(
                                   range = cp.CALITP_CATEGORY_BRIGHT_COLORS + 
                                   cp.CALITP_CATEGORY_BOLD_COLORS
                               ))
    ).properties(
    title=f"Total Annual Unlinked Passenger Trips per Reporter in RTPA since 2018",
    width = WIDTH, 
    height = HEIGHT,
    ).resolve_scale(y="independent").interactive()
    
    return bar_chart

### Report Totals

In [15]:
Markdown(f"""
Within {rtpa}:
- Number of Reporters: <b>{agency_count}</b>.
- Total Unlinked Passenger Trips since the beginning of this report: <b>{total_upt:,}</b>.
- Individual Reporters ridership breakdown:
""")


Within Metropolitan Transportation Commission:
- Number of Reporters: <b>32</b>.
- Total Unlinked Passenger Trips since the beginning of this report: <b>2,009,957,677.0</b>.
- Individual Reporters ridership breakdown:


In [16]:
display(agency_agg_yr.reset_index(drop=True))

,agency_name,total_upt,pct_of_total_upt
0,City and County of San Francisco (SFMTA) - Tra...,922569306,45.90
1,San Francisco Bay Area Rapid Transit District ...,455096497,22.64
2,Alameda-Contra Costa Transit District,238095061,11.85
3,Santa Clara Valley Transportation Authority (VTA),156318886,7.78
4,Peninsula Corridor Joint Powers Board (PCJPB),63666313,3.17
5,San Mateo County Transit District (SMCTD),52923694,2.63
6,"Golden Gate Bridge, Highway and Transportation...",20383349,1.01
7,Central Contra Costa Transit Authority (CCCTA),15567938,0.77
8,Marin County Transit District (MCTD),15158172,0.75
9,San Francisco Bay Area Water Emergency Transpo...,11893820,0.59


In [17]:
tooltip_list = ["agency_name","total_upt","pct_of_total_upt"]

total_upt_chart(
    agency_agg_yr,
    x_col="agency_name", 
    y_col="total_upt", 
    tool_tip=tooltip_list)

alt.Chart(...)

### Agency

In [32]:
make_line_chart(
    by_agency_long, 
    y_col = "upt", 
    color_col = "agency_name",
    title="Total Annual Unlinked Passenger Trips by Reporter"
)


alt.FacetChart(...)

In [34]:
make_bar_chart(
    by_agency_long, 
    y_col = "change_1yr", 
    color_col = "agency_name",
    title= "Yearly Change in Unlinked Passenger Trips by Agency"
)

alt.FacetChart(...)

## Transit Mode

In [20]:
make_line_chart(by_mode_long, y_col = "upt", color_col = "mode", title = "Total Annual UPT by Mode")

alt.FacetChart(...)

In [21]:
make_bar_chart(by_mode_long, y_col = "change_1yr", color_col = "Yearly Change in UPT by Mode")

alt.FacetChart(...)

## Type of Service

In [22]:
make_line_chart(by_tos_long, y_col = "upt", color_col = "service", title= "Total Annual UPT by Type of Service")

alt.FacetChart(...)

In [23]:
make_bar_chart(by_tos_long, y_col = "change_1yr", color_col = "service", title = "Total Change in UPT by Type of Service")

alt.FacetChart(...)

## Reporter Type

In [36]:
make_line_chart(by_reporter_type_long, y_col = "upt", color_col = "reporter_type", title= "Total Annual UPT by NTD Reporter Type")

alt.FacetChart(...)

In [38]:
make_bar_chart(by_reporter_type_long, y_col = "change_1yr", color_col = "reporter_type", title= "Total Annual UPT by NTD Reporter Type")

alt.FacetChart(...)